In [3]:
#importing the dataset
import pandas as pd
col_list = ["Column2", "Column3"]
col_list2 = ["Column2"]
dataset = pd.read_excel("Training.data.xlsx", usecols=col_list)
testnset = pd.read_excel("Test.data.xlsx", usecols=col_list2)

                                                Column2   Column3
0     Waiting for my mind to have a breakdown once t...  moderate
1     My new years resolution : I'm gonna get my ass...  moderate
2     New year : Somone else Feeling like 2020 will ...  moderate
3     My story I guess : Hi, Im from Germany and my ...  moderate
4     Sat in the dark and cried myself going into th...  moderate
...                                                 ...       ...
8886  Ways to reverse memory loss from depression? :...    severe
8887  A Comprehensive Guide To Slowly Getting Better...    severe
8888  I don’t think college is right for me : TW: su...    severe
8889  Please help: Severe insomnia affecting me in m...    severe
8890  With each passing day my depression is getting...    severe

[8891 rows x 2 columns]


In [5]:
# cleaning the text : deleting the emojis, the existing emails, the punctuation, the present digits,
# the hyperlinks and the stopwords (a,the,is, etc)
import nltk
import openpyxl
from nltk.corpus import stopwords, words, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re, string
from string import punctuation, digits
def delete_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", re.UNICODE)
    return emoji_pattern.sub(r'', text)  # no emoji


def delete_digits(text):
    text = text.lower()
    clean = text.translate(str.maketrans('', '', digits))
    return clean


def delete_punctuation(text):
    clean = text.translate(str.maketrans('', '', punctuation + '’“”'))
    return clean


stop = set(stopwords.words('english'))
punct = list(string.punctuation)
stop.update(punct)


def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)


def remove_hyperlinks(text):
    for word in text:
        if re.match(r'^http', word):
            text.remove(word)
    return " ".join(text)


def remove_emails(text):
    text = text.split()
    for i in text:
        if '@' in i.strip().lower():
            text.remove(i)
    return " ".join(text)


def denoise_text(text):
    text = remove_emails(text)
    text = delete_punctuation(text)
    text = delete_emoji(text)
    text = delete_digits(text)
    text = remove_stopwords(text)
    text = text.split()
    text = remove_hyperlinks(text)
    return text


dataset['Column2'] = dataset['Column2'].apply(denoise_text)
print(dataset['Column2'][0])

waiting mind breakdown new year feeling isnt anymore dont know anyone else im little bit worried ill go back depressed days time something last year tried breakdowns start mere days later broke crying wasnt entire year december ok month wait weird way act feel feels bit normal


In [6]:
# lemmatization : crying -> cry, days -> day
def get_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lemm(text):
    text = text.split()
    lemmatizer = WordNetLemmatizer()
    sar_list_lemmatizer = [lemmatizer.lemmatize(word, get_pos(word)) for word in text]
    return " ".join(sar_list_lemmatizer)

print(lemm(dataset['Column2'][0]))

wait mind breakdown new year feel isnt anymore dont know anyone else im little bit worried ill go back depressed day time something last year try breakdown start mere day later broke cry wasnt entire year december ok month wait weird way act feel feel bit normal


In [7]:
# CountVectorizer reprezentation for the user tweets
from sklearn.feature_extraction.text import CountVectorizer
x_train = dataset['Column2']
vectorizer = CountVectorizer()
vectorizer.fit(x_train)
x_train_dtm = vectorizer.transform(x_train)

x_train_dtm = vectorizer.fit_transform(x_train)
# print(x_train_dtm)

  (0, 12552)	1
  (0, 7222)	1
  (0, 1397)	1
  (0, 7593)	1
  (0, 12994)	3
  (0, 4256)	1
  (0, 6149)	1
  (0, 541)	1
  (0, 3353)	1
  (0, 6384)	1
  (0, 545)	1
  (0, 3663)	1
  (0, 5726)	1
  (0, 6671)	1
  (0, 1177)	2
  (0, 12906)	1
  (0, 5715)	1
  (0, 4929)	1
  (0, 872)	1
  (0, 2936)	1
  (0, 2744)	2
  (0, 11694)	1
  (0, 10616)	1
  (0, 6447)	1
  (0, 11922)	1
  :	:
  (8890, 4993)	2
  (8890, 11516)	2
  (8890, 5020)	1
  (8890, 5000)	1
  (8890, 1337)	1
  (8890, 537)	1
  (8890, 273)	1
  (8890, 5228)	1
  (8890, 10625)	1
  (8890, 7651)	1
  (8890, 222)	1
  (8890, 11532)	1
  (8890, 7)	1
  (8890, 5281)	2
  (8890, 5375)	1
  (8890, 3476)	1
  (8890, 9090)	1
  (8890, 10139)	1
  (8890, 5209)	1
  (8890, 2025)	1
  (8890, 6319)	2
  (8890, 6481)	1
  (8890, 2184)	1
  (8890, 10162)	1
  (8890, 1808)	1
